# Imports
#### Pandas
Data manipulation and analysis

#### MatPlotLib Pyplot
2D plotting

#### Datetime
Dates and time

#### Numpy
Supports large, multi-dimensional arrays and matrix manipulation and high level mathematical functions on these arrays

#### SQLAlchemy
Database Import

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import numpy as np
from sqlalchemy import create_engine

# Import db pw
from keys import pw

## Reusable References

In [ ]:
BOLD = '\033[1m'
END = '\033[0m'

### Create the Database Connection

In [ ]:
#db_uri = 'postgresql://postgres:' + pw + '@localhost:5432/SQLChallenge'
#engine = create_engine(db_uri)
#connection = engine.connect()

# This code will be replaced with accessing the database

In [ ]:
# Import Employees, Titles and Salaries csv's
emp_csv = './data/employees.csv'
empDateCols = ['birth_date', 'hire_date']
employees = pd.read_csv(emp_csv, parse_dates=empDateCols)

title_csv = './data/titles.csv'
titleDateCols = ['from_date', 'to_date']
titles = pd.read_csv(title_csv, parse_dates=titleDateCols)

sal_csv = './data/salaries.csv'
salaryDateCols = ['from_date', 'to_date']
salaries = pd.read_csv(sal_csv, parse_dates=salaryDateCols)

# Create a bar chart of average salary by title
#### Assumption
* Only include current employee salaries since historic salaries do not represent current day value

### Start by selecting only current employees

In [ ]:
# Start with the salaries table and select only rows that contain a to_date year of 9999 (indicates current employee)
# First, verify the data types of the to_date column
print(BOLD + 'The to_date column of the salary dataframe is of type:' + END)
print(type(salaries['to_date'].iat[0]))

# Alternatively we can look at all columns at one time
print(BOLD + '\n\nThe data types in the salary dataframe are:' + END)
print(salaries.dtypes)

# Now lets check value_counts
print(BOLD + '\n\nThe value counts for the to_date column are:' + END)
print(salaries['to_date'].value_counts().sort_values(ascending=False))

print(BOLD + 'NOTE: There are no values for the year 9999, so we cannot use this column to determine current employees.' + END)
# We can confirm, however that there are no values by creating a dataframe as well
current_salary = salaries.loc[salaries['to_date'].dt.year==9999,:]
print(BOLD + '\n\nThe results of searching for to_date values in the year 9999 are as expected with no returned values:' + END)
print(current_salary)

print(BOLD + 'NOTE:  The dataframe is empty which aligns with our expectations' + END)

In [ ]:
# Results above show that not all to_date columns in this database use the same convention
# Instead, lets check the titles table

# we can look at all columns at one time to check for datatypes
print(BOLD + 'The data types in the titles dataframe are:' + END)
print(titles.dtypes)
print(BOLD + 'NOTE:  The to_date column would not convert to datetime' + END)

# Now lets check value_counts
print(BOLD + '\n\nThe value counts for the to_date column are:' + END)
print(titles['to_date'].value_counts(ascending=True))
print(BOLD + 'NOTE:  There are roughly 24k employees currently.' + END)

# NOTE - The titles table to_date column will not come in as a date, therefore we perform a string compare
print(BOLD + "\n\nTitles dataframe filtered for current employees (to_date = 9999-01-01):" + END)
titles['to_date'].value_counts(ascending=False)
current_title = titles.loc[titles['to_date'] == '9999-01-01',:]
current_title.head()

### Now, merge the titles (current employees), employees and salaries dataframes

In [ ]:
# Start with merging employees to current titles
print(BOLD + 'Employees merged with Titles (current employees only)' + END)
emp_title = employees.merge(current_title, on='emp_no')
emp_title.head()

In [ ]:
# Now merge the salaries
print(BOLD + 'Salaries merged with Employees and Titles (current employees only)' + END)
emp_title_salary = emp_title.merge(salaries, on='emp_no').sort_values(['title'], ascending=True).reset_index()

# Finally, let's select only the columns we need
emp_title_salary = emp_title_salary[['title', 'salary']]
emp_title_salary.head()

### Next, groupby title and create statistics used for graphing

In [ ]:
# groupby title and create relevant statistics on the salary
print(BOLD + 'Salary statistics by Title' + END)
statistics_salary_by_title = emp_title_salary.groupby(['title']).aggregate(
    {
        'salary':['mean', 'median', 'min', 'max', 'count', 'sem']
    }
).sort_values(by=['title'], ascending=True)
statistics_salary_by_title.columns = statistics_salary_by_title.columns.droplevel(0)
statistics_salary_by_title.sort_values(['title'], ascending=True).reset_index()


### Create a Bar Chart to Show Average Salary by Job Title (and include error bars)

In [ ]:
# Create plot
fig, ax = plt.subplots(figsize=(14,8))
plt.bar(statistics_salary_by_title.index, statistics_salary_by_title['mean'], color='green',
        alpha=0.5, yerr=statistics_salary_by_title['sem'], label='Average Salary ($)')

# Add labels and formatting
plt.xlabel("Job Title", weight='bold', size=12)
plt.ylabel("Annual Salary ($)", weight='bold', size=12)
plt.title("Annual Salary by Job Title", weight='bold', size=22)
plt.xticks(statistics_salary_by_title.index, rotation=90)
plt.legend(loc='best')
plt.grid(alpha=0.2)

# Save the chart out
plt.savefig("./Output/SalaryByTitleBar.png")

# Show plot
plt.show()

### Interesting... why does Manager have such large error bars?
Let's dig a little more and plot the distributions via a box plot

#### First, create a function

In [ ]:
# Function comparing populations by Boxplots
def boxPlotCompare(srStaff, staff, mgr, techLdr, eng, srEng, asstEng, title):
    
    # Set the figure size
    fig = plt.figure(figsize=(14,8))
    axBox = fig.add_subplot()
    
    # Show box plots of the data
    box_plot_data=[srStaff, staff, mgr, techLdr, eng, srEng, asstEng]
    plt.boxplot(box_plot_data)
    
    # Format the chart
    plt.title(title, color='k', size=24, weight='bold')
    plt.xticks([1, 2, 3, 4, 5, 6, 7], ['Assistant Engineer', 'Engineer', 'Manager', 'Senior Engineer', 'Senior Staff', 'Staff', 'Technique Leader'])    
    plt.xlabel("Job Title", size=14, weight='bold')
    plt.ylabel("Salary ($)", size=14, weight='bold')

    # Save the chart out
    plt.savefig("./Output/SalaryByTitleBoxPlot.png")

    # Show the chart
    plt.show()
       
    return

#### Next, create a series for each job title

In [ ]:
# Create a salary series for each title
srStaff = emp_title_salary.loc[(emp_title_salary['title']=='Senior Staff'),'salary']
staff = emp_title_salary.loc[(emp_title_salary['title']=='Staff'),'salary']
mgr = emp_title_salary.loc[(emp_title_salary['title']=='Manager'),'salary']
techLdr = emp_title_salary.loc[(emp_title_salary['title']=='Technique Leader'),'salary']
eng = emp_title_salary.loc[(emp_title_salary['title']=='Engineer'),'salary']
srEng = emp_title_salary.loc[(emp_title_salary['title']=='Senior Engineer'),'salary']
asstEng = emp_title_salary.loc[(emp_title_salary['title']=='Assistant Engineer'),'salary']

#### Finally, plot the box plots

In [ ]:
# Plot salaries by title in box plots
boxPlotCompare(asstEng, eng, mgr, srEng, srStaff, staff, techLdr, "Salaries ($) by Job Title - Current Employed")

#### Manager has a small range (25 percentile to 75 percentile) and the fewest outliers, but the largest errors.
Let's dig more...

#### This time, let's look at a scatter plot of the individual data and plot the median as a reference

In [ ]:
# Plot the data
fig = plt.figure(figsize=(14,8))
ax = fig.add_subplot()
plt.scatter(emp_title_salary["title"], emp_title_salary["salary"], c="b", alpha=0.5, label='salary')
plt.plot(statistics_salary_by_title.index, statistics_salary_by_title['median'], color='green', label='median')

# format the chart
plt.title(f'Salary ($) vs Job Title - Currently Employed', color='k', size=24, weight='bold')
plt.xlabel('Job Title', weight='bold', size=12)
plt.ylabel('Salary ($)', weight='bold', size=12)
plt.grid(alpha=0.2)
plt.legend(loc='best')

# Save the chart out
plt.savefig("./Output/SalaryByTitleScatterAndMedian.png")

# Show the plot
plt.show()


#### Interesting, there is very little data for Manager so there is a higher spread between points
Let's now check how many data points there are

In [ ]:
# Redisplay the statistics table
print(BOLD + 'Statistics of Salaries by Job Title' + END)
print(statistics_salary_by_title)

print(BOLD + 'NOTE:  Manager only has 9 data points compared to thousands for the other titles.' + END)

## Now, per the last item requested by the boss, look up employee id

#### Look up Employee ID = 499942

In [ ]:
my_info = employees.loc[employees["emp_no"] == 499942, :]
my_info

#### NOTE the fact that suspicians were correct and the data is fake.